### This notebook will get the Airline Data from the Azure Blob Storage and stores in the Table in the Databricks Environment

### Step 1: Set the data location and type

To get started, we need to set the location and type of the file.

In [7]:
%%pyspark
adls_storage_name = "sparkpoolstorage"
adls_storage_access_key = "sbwpC939Y+Z65urA1yWYTj5bndsUCpmNIh97d7so03IZUZJ15rTCkTuEAGrx3C+ccGKLa8UD3/8E+AStg0J7OQ=="

StatementMeta(airlinespkpool, 1, 7, Finished, Available)

In [9]:
%%pyspark
file_location = "wasbs://airline@sparkpoolstorage.blob.core.windows.net/airlinedata"
file_type = "csv"

StatementMeta(airlinespkpool, 1, 9, Finished, Available)

In [10]:
%%pyspark
spark.conf.set(
  "fs.azure.account.key."+adls_storage_name+".blob.core.windows.net", adls_storage_access_key)

StatementMeta(airlinespkpool, 1, 10, Finished, Available)

### Step 2: Read all the airline data present under the folder airport

Now that we have specified our file metadata, we can create a DataFrame. Notice that we use an *option* to specify that we want to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python.

In [11]:
%%pyspark
df_air = spark.read.format(file_type).option("inferSchema", "true").option("header", "true").load(file_location)

StatementMeta(airlinespkpool, 1, 11, Finished, Available)

### Step 3: Query the data

Now that we have created our DataFrame, we can query it. For instance, you can identify particular columns to select and display.

In [12]:
%%pyspark
display(df_air)

StatementMeta(airlinespkpool, 1, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, d4e9df21-76aa-4627-ab52-464c2624bd5c)

### Step 4: Create a temporary view or table for analysing the data

If you want to query this data as a table, you can simply register it as a *view* or a table.

In [13]:
%%pyspark
df_air.createOrReplaceTempView("airlineTemp")

StatementMeta(airlinespkpool, 1, 13, Finished, Available)

We can query this view using Spark SQL. For instance, we can perform a simple aggregation. Notice how we can use `%sql` to query the view from SQL.

In [21]:
%%sql
SELECT * FROM airlineTemp WHERE TAIL_NUM = 'N743SW'

StatementMeta(airlinespkpool, 1, 21, Finished, Available)

<Spark SQL result set with 2 rows and 33 fields>

In [22]:
%%pyspark
spark.sql("CREATE DATABASE IF NOT EXISTS airlineDB")
df_air.write.mode("overwrite").saveAsTable("airlineDB.airlineReporting")

StatementMeta(airlinespkpool, 1, 22, Finished, Available)

### Step 5: Create a Permanent Table

Since this table is registered as a temp view, it will be available only to this notebook. If you'd like other users to be able to query this table, we have to create a permanent table from the DataFrame as below.

In [23]:
%%pyspark
df_air.write.format("csv").saveAsTable("airlineDB.airlineReportingdup")

StatementMeta(airlinespkpool, 1, 23, Finished, Available)

In [24]:
%%sql
Select * from airlineDB.airlineReporting
WHERE TAIL_NUM = 'N743SW'

StatementMeta(airlinespkpool, 1, 24, Finished, Available)

<Spark SQL result set with 2 rows and 33 fields>

In [25]:
%%sql
Select count(1) from airlineDB.airlineReportingdup

StatementMeta(airlinespkpool, 1, 25, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

#### This table will persist across cluster restarts and allow various users across different notebooks to query this data.